In [1]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_9d3ba0df94c74c0abac44d55ebf2a601 = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_9d3ba0df94c74c0abac44d55ebf2a601 = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_9d3ba0df94c74c0abac44d55ebf2a601 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='jG7rSfw-gAgJCeEk0GrsfFMMZsDNDuMusb6h_KWOHGG4',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_9d3ba0df94c74c0abac44d55ebf2a601)

body = client_9d3ba0df94c74c0abac44d55ebf2a601.get_object(Bucket='rainfallprediction-donotdelete-pr-ltz8foccv0x3dv',Key='weatherAUS.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)
data.head()


,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [2]:
# removing columns with more than 20% missing values and segregatting cat and num variables
data_cat = data[['RainToday','WindGustDir','WindDir9am','WindDir3pm']]
data.drop(columns=['Evaporation','Sunshine','Cloud9am','Cloud3pm'],axis=1,inplace=True)
data.drop(columns=['RainToday','WindGustDir','WindDir9am','WindDir3pm'],axis=1,inplace=True)

In [3]:
# filling the missing data of numeric variables with mean
data['MinTemp'].fillna(data['MinTemp'].mean(),inplace=True)
data['MaxTemp'].fillna(data['MaxTemp'].mean(),inplace=True)
data['Rainfall'].fillna(data['Rainfall'].mean(),inplace=True)
data['WindGustSpeed'].fillna(data['WindGustSpeed'].mean(),inplace=True)
data['WindSpeed9am'].fillna(data['WindSpeed9am'].mean(),inplace=True)
data['WindSpeed3pm'].fillna(data['WindSpeed3pm'].mean(),inplace=True)
data['Humidity9am'].fillna(data['Humidity9am'].mean(),inplace=True)
data['Humidity3pm'].fillna(data['Humidity3pm'].mean(),inplace=True)
data['Pressure9am'].fillna(data['Pressure9am'].mean(),inplace=True)
data['Pressure3pm'].fillna(data['Pressure3pm'].mean(),inplace=True)
data['Temp9am'].fillna(data['Temp9am'].mean(),inplace=True)
data['Temp3pm'].fillna(data['Temp3pm'].mean(),inplace=True)

In [4]:
# loading the names of categorical columns
cat_names = data_cat.columns

In [5]:
# intializing the simple imputer for missing categorical values
import numpy as np
from sklearn.impute import SimpleImputer
imp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [6]:
# fitting and transforming the missing data
data_cat = imp_mode.fit_transform(data_cat)

In [7]:
# converting array to dataframe
data_cat = pd.DataFrame(data_cat,columns=cat_names)

In [8]:
# concatinating the categorical and numeric data
data = pd.concat([data,data_cat],axis=1)

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [10]:
#fitting and transforming the categorical data
data['Location'] = le.fit_transform(data['Location'])
data['RainToday'] = le.fit_transform(data['RainToday'])
data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
data['WindDir9am'] = le.fit_transform(data['WindDir9am'])
data['WindDir3pm'] = le.fit_transform(data['WindDir3pm'])

In [11]:
# spliiting the date column into year,month,day
data[["year", "month", "day"]] = data["Date"].str.split("-", expand = True)

In [12]:
# removing the main column
data.drop(['Date'],axis=1,inplace=True)

In [13]:
data.dropna(axis=0,how='any',inplace=True) #output has missing values,so removing the rows with missing values

In [14]:
from sklearn.preprocessing import StandardScaler

In [15]:
#splitting x and y values
y = data['RainTomorrow']
x = data.drop('RainTomorrow',axis=1)

In [16]:
names = x.columns # loading the names of the x_features

In [17]:
sc = StandardScaler() # initializing the standardscaler

In [18]:
x = sc.fit_transform(x) # fitting an transforming the data into standard

In [19]:
x = pd.DataFrame(x,columns=names) #scaled data turns into array format,converting them into dataframe

In [20]:
from sklearn import model_selection

In [21]:
x_train,x_test,y_train,y_test = model_selection.train_test_split(x,y,test_size =0.2,random_state =0)

In [22]:
from sklearn import ensemble
from sklearn import metrics

In [23]:
model = ensemble.RandomForestClassifier(criterion='entropy',max_features='sqrt',n_estimators = 25)

In [24]:
model.fit(x_train,y_train)
y_tp= model.predict(x_train)
metrics.accuracy_score(y_train,y_tp)

0.9983648926631151

In [25]:
y_pred= model.predict(x_test)
metrics.accuracy_score(y_test,y_pred)

0.8555153134779704

In [26]:
import ibm_watson_machine_learning

In [27]:
from ibm_watson_machine_learning import APIClient

In [28]:
wml_credntials = {
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"ZwTq99YvAP3KVVNcwyRggCwq0LapRgKdnJKPiboMqaR9"
}

In [29]:
client = APIClient(wml_credntials)

In [30]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [31]:
space_uid = guid_from_space_name(client,space_name="Rainfall_prediction")
print("Space_UID = " + space_uid)

Space_UID = 0d3327a9-9d61-4630-a7c0-1cabea67f67c


In [32]:
client.set.default_space(space_uid)

'SUCCESS'

In [33]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
pytorch-onnx_1.3-py3.6         1bc6029

In [34]:
#setting the python version
software_spec_uid = client.software_specifications.get_uid_by_name("default_py3.8")
software_spec_uid

'ab9e1b80-f2ce-592c-a7d2-4f2344f77194'

In [35]:
MODEL_NAME = 'predicting the rainfall occurance'
DEPLOYMENT_NAME = 'Rainfall_prediction'
Rainfall_MODEL = model

In [36]:
import sklearn
sklearn.__version__

'0.23.2'

In [37]:
# Setup model meta
model_props = {
    client.repository.ModelMetaNames.NAME: MODEL_NAME, 
    client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.23', 
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid 
}


In [38]:
# saving the model into model details
model_details = client.repository.store_model(model=Rainfall_MODEL, meta_props=model_props,training_data=x_train, training_target=y_train)

In [39]:
model_uid = client.repository.get_model_uid(model_details)

In [40]:
model_uid

'11d9d232-ed24-40be-ba49-33e138635d6c'

In [47]:
x_test.iloc[1,:]

Location        -1.246055
MinTemp         -0.843092
MaxTemp         -1.241541
Rainfall        -0.184026
WindGustSpeed    0.457621
WindSpeed9am     0.338674
WindSpeed3pm     0.385479
Humidity9am      0.589252
Humidity3pm      0.171243
Pressure9am     -1.104823
Pressure3pm     -0.427728
Temp9am         -1.017828
Temp3pm         -1.308033
RainToday       -0.532962
WindGustDir     -0.012577
WindDir9am       1.107432
WindDir3pm       0.046525
year            -0.692151
month           -0.409324
day              0.941669
Name: 57576, dtype: float64

In [48]:
x_test.head(1)

,Location,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,WindGustDir,WindDir9am,WindDir3pm,year,month,day
86232,-1.105581,0.816035,0.812041,-0.279002,-0.531849,-0.339212,0.041502,-0.308698,0.46347,-0.674953,-0.951542,1.407953,0.889691,-0.532962,-0.651657,0.442931,-0.613352,-0.298643,1.049894,-0.763241


In [ ]:
[-1.105581,0.816035,0.812041,-0.279002,-0.531849,-0.339212,0.041502,-0.308698,0.46347,-0.674953,-0.951542,1.407953,0.889691,-0.532962,-0.651657,0.442931,-0.613352,-0.298643,1.049894,-0.763241]

In [49]:
x_test.columns

Index(['Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'WindGustSpeed',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Temp9am', 'Temp3pm', 'RainToday',
       'WindGustDir', 'WindDir9am', 'WindDir3pm', 'year', 'month', 'day'],
      dtype='object')